# Data Description

## Data Source

* PUBG Match Deaths and Statistics, Kaggle 
    \- https://www.kaggle.com/skihikingkevin/pubg-match-deaths

## Data Introduction

In this Kaggle Dataset, over 720,000 competitive matches from the popular game PlayerUnknown's Battlegrounds. The data was extracted from pubg.op.gg, a game tracker website.


### PlayerUnknown's Battlegrounds

PUBG is a first/third-person shooter battle royale style game that matches over 90 players on a large island where teams and players fight to the death until one remains. Players are airdropped from an airplane onto the island where they are to scavenge towns and buildings for weapons, ammo, armor and first-aid. Players will then decide to either fight or hide with the ultimate goal of being the last one standing. A bluezone (see below) will appear a few minutes into the game to corral players closer and closer together by dealing damage to anyone that stands within the bluezone and sparing whoever is within the safe zone.


### The Dataset

This dataset provides two zips: aggregate and deaths.

In **deaths**, the files record every death that occurred within the 720k matches. That is, each row documents an event where a player has died in the match.

In **aggregate**, each match's meta information and player statistics are summarized (as provided by pubg). It includes various aggregate statistics such as player kills, damage, distance walked, etc as well as metadata on the match itself such as queue size, fpp/tpp, date, etc.
The uncompressed data is divided into 5 chunks of approximately 2gb each.

### Columns in deaths

1. killed_by: Which weapon is killed
1. killer_name: Killer game id
1. killer_placement: The final ranking of the team where the killer is located
1. killer_position_x: X coordinate of the killer when the killing behavior occurs
1. killer_position_y: Y coordinate of the killer when the killing behavior occurs
1. map: Game Map(Erangel island/ Miramar desert)
1. match_id : Event Unique ID
1. time: When the kill occurs(How many seconds after the game starts)
1. victim_name: The killed game id
1. victim_position_x: X coordinate of the person being killed when the killing occurs
1. victim_position_y: Y coordinate of the killer at the time of the killing behavior

### Columns in aggregate

1. date: Start time of the game
1. game_size: Site size
1. match_id: Event Unique ID
1. match_mode: Game Mode(First/ Third Person View)
1. party_size: Squad size(1person/ 2people/ 4people)
1. player_assists: Rescue teammates
1. player_dbno: Number of times the player was knocked down
1. player_dist_ride: Driving Distance
1. player_dist_walk: Walking distance
1. player_dmg: Injury points
1. player_kills: kills
1. player_name: Player Game id
1. player_survive_time: Player survival time
1. team_id: The player's team number
1. team_placement: The final ranking of the player's team

# 라이브러리 및 데이터 로드

## 라이브러리

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from tqdm.auto import tqdm
tqdm.pandas()
import os

In [2]:
# MAC 기준
# Window font: 'Malgun Gothic'

plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)

In [3]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## 데이터

In [ ]:
data_dir = '../dataset/raw/'

In [ ]:
def data_load(data_dir, name_list, df_list):
    df_list = []
    for i in tqdm(name_list):
        df_list.append(pd.read_csv(data_dir + i))
    return df_list

In [ ]:
agg_data_dir = '../dataset/raw/aggregate/'
agg_name_list = sorted(os.listdir(agg_data_dir))
agg_list = []
agg_list = data_load(agg_data_dir, agg_name_list, agg_list)

In [ ]:
deaths_data_dir = '../dataset/raw/deaths/'
deaths_name_list = sorted(os.listdir(deaths_data_dir))
deaths_list = []
deaths_list = data_load(deaths_data_dir, deaths_name_list, deaths_list)

In [ ]:
def get_shape(df_list):
    for i in df_list:
        print(i.shape)

In [ ]:
get_shape(agg_list)

In [ ]:
get_shape(deaths_list)

# Data preprocessing

## Aggregate의 na 제거

In [ ]:
def df_drop_na(df_list):
    for i in tqdm(range(len(df_list))):
        df_list[i] = df_list[i].dropna()

In [ ]:
df_drop_na(agg_list)

## match_mode 제거
* 모두 'tpp'만 가짐

In [ ]:
def del_col(df_list, col_name):
    for i in tqdm(df_list):
        del i[col_name]

In [ ]:
del_col(agg_list, 'match_mode')

## 두 데이터에서 매칭되지 않는 match_id 제거

In [ ]:
def get_unique_match_id(df_list):
    match_id = []
    for i in df_list:
        match_id += [x for x in i['match_id'].unique()]
    return match_id

In [ ]:
agg_match_id = get_unique_match_id(agg_list)

In [ ]:
len(agg_match_id)

In [ ]:
deaths_match_id = get_unique_match_id(deaths_list)

In [ ]:
agg_mat = set(agg_match_id)
deaths_mat = set(deaths_match_id)

In [ ]:
len(agg_mat), len(deaths_mat), len(agg_mat & deaths_mat)

In [ ]:
get_shape(deaths_list)

In [ ]:
for i in range(len(deaths_list)):
    deaths_list[i] = deaths_list[i][deaths_list[i]['match_id'].isin(agg_match_id)]

In [ ]:
get_shape(deaths_list)

## deaths data na 처리

### map na 처리

In [ ]:
deaths_list[0]['map'].unique()

* map 결측값 대체 가능 여부 확인

In [ ]:
map_na_match_id = []
for i in deaths_list:
    map_na_match_id += [x for x in i.loc[i['map'].isnull(), 'match_id'].unique()]

In [ ]:
E_match_id = []
for i in deaths_list:
    E_match_id += [x for x in i.loc[i['map'] == 'ERANGEL', 'match_id'].unique()]

In [ ]:
M_match_id = []
for i in deaths_list:
    M_match_id += [x for x in i.loc[i['map'] == 'MIRAMAR', 'match_id'].unique()]

* 겹치는 match_id 확인

In [ ]:
map_na_match_id = set(map_na_match_id)
E_match_id = set(E_match_id)
M_match_id = set(M_match_id)

In [ ]:
len(map_na_match_id & E_match_id), len(map_na_match_id & M_match_id), len(E_match_id & M_match_id) 

* 대체 불가능 판단 -> Drop

In [ ]:
get_shape(deaths_list)

### deaths 데이터 na drop

In [ ]:
df_drop_na(deaths_list)

In [ ]:
get_shape(deaths_list)

## Data 병합

* key columns
    * agg.match_id = deaths.match_id
    * agg.player_name = deaths.killer_name

In [ ]:
# key column의 이름을 맞춰야 함
# deaths의 killer_name을 player_name으로 변경

def chg_col_names(df_list, col_names):
    for i in tqdm(df_list):
        i.columns = col_names

In [ ]:
deaths_list[0].columns

In [ ]:
deaths_cols = ['killed_by', 'player_name', 'killer_placement', 'killer_position_x',
               'killer_position_y', 'map', 'match_id', 'time', 'victim_name',
               'victim_placement', 'victim_position_x', 'victim_position_y']

In [ ]:
# key column 이름 맞추기

chg_col_names(deaths_list, deaths_cols)

## Aggregate 데이터와 Deaths 데이터 Merge

* 같은 번호의 데이터와 짝을 이루는 것을 match_id로 확인함
    * e.g. agg_0는 deaths_0과 Merge

In [ ]:
get_shape(agg_list)

In [ ]:
get_shape(deaths_list)

In [ ]:
deaths_list[0]['map'].unique()

In [ ]:
def get_merged_df(left, right, join, keys):
    df_list = []
    for i in tqdm(range(len(left))):
        df_list.append(pd.merge(left[i], right[i], how=join, on=keys))
    return df_list

In [ ]:
df_merge = get_merged_df(agg_list, deaths_list, 'left', ['match_id', 'player_name'])

In [ ]:
get_shape(df_merge)

## map 채우기

In [ ]:
def fill_map_na(df_list, match_id_list, map_name):
    for i in df_list:
        i.loc[(i['match_id'].isin(match_id_list))&(i['map'].isnull()), 'map'] = map_name
    return df_list

In [ ]:
match_id = [E_match_id, M_match_id]
map_names = ['ERANGEL', 'MIRAMAR']

for i in tqdm(range(len(match_id))):
    df_merge = fill_map_na(df_merge, match_id[i], map_names[i])

## map별로 데이터 나누기

* ERANGEL과 MIRAMAR로 데이터 셋을 나눔

In [ ]:
def get_df_map(df_list, map_name):
    df_map = []
    for i in df_list:
        df_map.append(i[i['map'] == map_name])
    return df_map

In [ ]:
df_map  = []
for i in tqdm(range(len(map_names))):
    df_map.append(get_df_map(df_merge, map_names[i]))

## party_size 별로 데이터 나누기

* party_size에 따라 Tier가 다르기 때문에 데이터를 분리함

In [ ]:
def get_df_party_size(df_list, party_size):
    df_party_size = []
    for i in df_list:
        df_party_size.append(i[i['party_size'] == party_size])
    return df_party_size

In [ ]:
df = []
party_size = [1, 2, 4]

for i in tqdm(range(len(party_size))):
    for j in range(len(df_map)):
        df.append(get_df_party_size(df_map[j], party_size[i]))

In [ ]:
# df[0]: solo, ERANGEL / df[1]: solo, MIRAMAR
# df[2]: duo, ERANGEL / df[3]: duo, MIRAMAR
# df[4]: squad, ERANGEL / df[5]: squad, MIRAMAR

len(df)

## party_size별 map별 dataset 합치기

In [ ]:
def get_concat(df_list):
    df_concat = []
    for i in tqdm(range(len(df_list))):
        df_concat.append(pd.concat(df_list[i], ignore_index=True))
    return df_concat

In [ ]:
# df_concat[0]: solo, ERANGEL / df_concat[1]: solo, MIRAMAR
# df_concat[2]: duo, ERANGEL / df_concat[3]: duo, MIRAMAR
# df_concat[4]: squad, ERANGEL / df_concat[5]: squad, MIRAMAR

df_concat = get_concat(df)

## killed_by 그룹핑

In [ ]:
def killed_by_refine(df):
    df['killed_by'] = df['killed_by'].replace({'death.WeapSawnoff_C': 'sawed_off', 
                                               'death.PlayerMale_A_C': 'Punch',
                                               'death.PG117_A_01_C': 'Boat' , 'death.RedZoneBomb_C': 'RedZone'})
    df['killed_by'] = df['killed_by'].replace(['Pickup Truck','Hit by Car','Buggy','Dacia','Motorbike',
                                               'Motorbike (SideCar)','Uaz','Van'], 'land_vehicle')
    df['killed_by'] = df['killed_by'].replace(['death.ProjMolotov_C', 'death.ProjMolotov_DamageField_C', 
                                               'death.Buff_FireDOT_C'], 'Molotov')
    df['killed_by'] = df['killed_by'].replace(['Aquarail','Boat'], 'water_vehicle')

In [ ]:
for i in tqdm(df_concat):
    killed_by_refine(i)

## csv로 내보내기

In [ ]:
def df_read_csv(df_list, data_dir, file_name):
    for i in tqdm(range(len(df_list))):
        df_list[i].to_csv(data_dir + f'{file_name[i]}.csv', index=False)

# Bug player 처리

In [4]:
data_dir = '../dataset/preprocessing/'

df = pd.read_csv(data_dir + 'squad_E.csv')

In [5]:
df_raw = df.copy()

In [6]:
class CheckingBugPlayer:
    
    '''
    Bug Player 처리를 위한 Class 입니다.
    아래의 기준 중 하나라도 만족하는 player는 Bug player로 지정합니다.
    
    dist_ride : 30000(30km) 초과
    dist_walk : 10000(10km) 초과
    kills :     30 kill 초과
    dmg :       3000 damage 초과
    kill_dist : 40000(400m) 초과
    dbno :      11 초과
    survive_time : 1900 초과
    
    '''
    
        
    def player_dist_ride(self, df):
        df.loc[df['player_dist_ride'] > 30000, 'outlier'] = 1
        return df
    
    def player_dist_walk(self, df):
        df.loc[df['player_dist_walk'] > 10000, 'outlier'] = 1
        return df
        
    def player_kills(self, df):
        df.loc[df['player_kills'] > 30, 'outlier'] = 1
        return df
    
    def player_dmg(self, df):
        df.loc[df['player_dmg'] > 3000, 'outlier'] = 1 
        return df
        
    def kill_dist(self, df):
        df['kill_dist'] = np.sqrt(((df['killer_position_x'] - df['victim_position_x']) ** 2) 
                                      + ((df['killer_position_y'] - df['victim_position_y']) ** 2))
        df['kill_dist'] = df['kill_dist'].fillna(0)
        df.loc[df['kill_dist'] > 40000, 'outlier'] = 1
        return df    
    
    def player_dbno(self, df):
        df.loc[df['player_dbno'] > 11, 'outlier'] = 1
        return df
    
    def player_survive_time(self, df):
        df.loc[df['player_survive_time'] > 1900, 'outlier'] = 1
        return df


## Bug player 처리

In [7]:
checker = CheckingBugPlayer()

df = checker.player_dist_ride(df)
df = checker.player_dist_walk(df)
df = checker.player_kills(df)
df = checker.player_dmg(df)
df = checker.kill_dist(df)
df = checker.player_dbno(df)
df = checker.player_survive_time(df)

df['outlier'] = df['outlier'].fillna(0)

In [8]:
df['outlier'].value_counts()

0.000    33914758
1.000     2316359
Name: outlier, dtype: int64

## 분석 Dataset 확인

In [9]:
df.describe()

,game_size,party_size,player_assists,player_dbno,player_dist_ride,player_dist_walk,player_dmg,player_kills,player_survive_time,team_id,team_placement,killer_placement,killer_position_x,killer_position_y,time,victim_placement,victim_position_x,victim_position_y,outlier,kill_dist
count,36231117.000,36231117.000,36231117.000,36231117.000,36231117.000,36231117.000,36231117.000,36231117.000,36231117.000,36231117.000,36231117.000,21775512.000,21775512.000,21775512.000,21775512.000,21775512.000,21775512.000,21775512.000,36231117.000,36231117.000
mean,26.975,4.000,0.455,1.806,1619.600,1676.894,255.247,2.151,18562.315,14.285,12.133,10.001,412058.181,395744.069,766.263,15.028,404669.838,388835.484,0.064,14278.071
std,1.807,0.000,0.843,2.557,2225.462,4419.428,334.019,3.108,33437725.829,8.070,8.135,7.791,149545.821,145386.571,557.284,7.754,157438.292,152740.310,0.245,84962.944
min,3.000,4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,1.000,-688028.000,-70411.010,23.000,1.000,-688028.000,-70481.330,0.000,0.000
25%,26.000,4.000,0.000,0.000,0.000,402.434,36.000,0.000,374.868,7.000,5.000,3.000,330982.575,291782.200,257.000,8.000,320256.425,289269.100,0.000,0.000
50%,27.000,4.000,0.000,1.000,453.057,1463.172,153.000,1.000,987.009,14.000,11.000,8.000,418874.300,385798.800,603.000,15.000,415837.400,380995.050,0.000,306.203
75%,28.000,4.000,1.000,3.000,2903.303,2617.174,349.000,3.000,1594.539,21.000,19.000,16.000,531930.100,490881.700,1256.000,22.000,526625.825,487756.650,0.000,2822.890
max,35.000,4.000,13.000,50.000,506175.400,1263531.380,6323.000,71.000,63648215000.000,58.000,35.000,35.000,810724.700,817629.800,2304.000,35.000,810775.600,817708.700,1.000,1080040.344


In [10]:
df_outlier = df.copy()

# 파생변수 생성 1

In [11]:
class DerivingVariables:
    
    '''
    파생변수를 만드는 Class 입니다.
    아래의 방법으로 파생변수를 생성하였습니다.
    
    date         : dtype을 datetime 으로 변환
    score        : (rank_point * 1) + (kill * 2) + (assist * 2)
    tier         : score column을 바탕으로 구함. squad의 +0.6은 위치를 맞추기 위한 수평이동
    KDA          : kill, assist, death를 이용해 파생변수 생성 
                   death는 match 합계에서 team_placement가 1등인 횟수를 제외하여 계산
    num_of_match : player별 총 게임 횟수를 계산
    
    '''
    
    def date(self, df):
        df.loc[:, 'date'] = pd.to_datetime(df['date'])
        return df
    
    def score(self, df):
        max_team_placement = df.groupby(['date', 'match_id'])['team_placement'].max().reset_index()
        max_team_placement.columns = ['date', 'match_id', 'max_placement']
        df = pd.merge(df, max_team_placement, how='left', on=['date', 'match_id'])
        
        df['each_game_score'] = (df['max_placement'] - df['team_placement'])*0.1 + np.log(df['player_dmg'] + 1) + df['player_dbno']*(-0.1)
        
        # score의 sum 
        each_score = df.groupby(['player_name', 'date', 'match_id'])['each_game_score'].mean().to_frame()
        total_score = each_score.groupby('player_name')['each_game_score'].sum().to_frame()
        total_score.columns = ['total_score']
        df['total_score'] = total_score.loc[df['player_name'], 'total_score'].values
        
        return df
    
    def kda_num_of_match(self, df):
        # 필요한 column 만 추출
        anal_col = df[['player_name', 'match_id','team_placement', 'player_kills', 'player_assists']]

        # num_of_match
        num_of_match = anal_col.groupby('player_name')['match_id'].nunique().reset_index()
        num_of_match.columns = ['player_name', 'num_of_match']

        # num_of_1st
        num_of_1st = anal_col[anal_col['team_placement'] == 1].groupby('player_name')['match_id'].nunique().reset_index()
        num_of_1st.columns = ['player_name', 'num_of_1st']

        # death 
        temp = num_of_match.merge(num_of_1st, how = 'left', on = 'player_name')
        temp = temp.fillna(0)
        temp.loc[:, 'deaths_sum'] = temp['num_of_match'] - temp['num_of_1st']
        death = temp[['player_name', 'deaths_sum']]

        # kill & assist
        KA_count = anal_col.groupby(['player_name', 'match_id'])[['player_kills', 'player_assists']].mean().reset_index()
        KA_sum = KA_count.groupby('player_name')[['player_kills', 'player_assists']].sum().reset_index()
        KA_sum.columns = ['player_name', 'kills_sum', 'assists_sum']

        # KDA
        KDA = KA_sum.merge(death, how = 'left', on = 'player_name')
        KDA.loc[:, 'kda'] = (KDA['kills_sum'] + KDA['assists_sum']) / KDA['deaths_sum']
        KDA_1 = KDA[['player_name', 'kda']]


        ## 여기 한번에 표현 할 수 있는가
        df = df.merge(KDA_1, how='left', on='player_name')
        df = df.merge(num_of_match, how='left', on='player_name' )

        return df

## 파생변수1 생성

In [12]:
variables = DerivingVariables()

df = variables.date(df)
df = variables.score(df)
df = variables.kda_num_of_match(df)

In [13]:
df_var1 = df.copy()

# 파생변수 생성 2

In [21]:
class DerivingVariables2:
    """
    
    
    """
    
    def date_vars(self, df):
        df['kor_date'] = df['date'] + datetime.timedelta(hours=9)
        df['day_of_week'] = df['kor_date'].dt.weekday
        df['weekend'] = [1 if x >= 6 else 0 for x in df['day_of_week'] ]
        df['date_time'] = df['kor_date'].dt.hour

        def get_time_zone(time):
            if time <= 4 :
                return 'peak time'
            elif time <= 10 :
                return 'morning'
            elif time <= 16 :
                return 'afternoon'
            elif time <= 22 :
                return 'evening'
            else:
                return 'peak time'

        df['time_zone'] = df['date_time'].apply(lambda x: get_time_zone(x))

        return df

    def weapon_category(self, df):

        def get_weapon(weapon):
            if weapon == 'water_vehicle|land_vehicle|Falling|Drown|RedZone|Bluezone':
                return 'accident'
            elif weapon == 'P18C|R1895|P92|P1911|S686|S1897|S12K|Sickle|Machete|Pan|Punch|Crowbar|Crossbow':
                return 'short_range'
            elif weapon == 'UMP9|Micro UZI|Tommy Gun|Vector|AUG|Groza|M249|DP-28':
                return 'middle_range'
            elif weapon == 'SKS|Mini 14|VSS|Mk14|Kar98k|M24|AWM':
                return 'long_range'
            elif weapon == 'Molotov|Grenade':
                return 'throwables'
            else:
                return weapon

        df['weapon'] = df['killed_by'].apply(lambda x: get_weapon(x))
        return df

    def dmg_per_kills(self, df):
        df['dmg_per_kills'] = df['player_dmg'] / df['player_kills']
        df.loc[(df['player_dmg'] == 0) & (df['player_kills'] != 0), 'dmg_per_kills'] = 0
        idx = df[df['dmg_per_kills'] == np.inf].index
        df.loc[idx, 'dmg_per_kills'] = df.loc[idx, 'player_dmg'] / 100000
        return df
    
    def ride_ratio(self, df):
        df['ride_ratio'] = df['player_dist_ride'] / df['player_dist_walk']
        df.loc[(df['player_dist_ride'] > 0) & (df['player_dist_walk'] == 0), 'ride_ratio'] = 10000
        return df

    def cumulative_sum(self, df):
        df_cum_grp = df.groupby(['date', 'match_id', 'player_name'])[['each_game_score', 'player_kills', 
                                                                          'player_dmg', 'player_assists',
                                                                          'dmg_per_kills', 
                                                                          'ride_ratio']].mean().reset_index()
        df_cum_grp['num'] = 1

        df_cum_sum = df_cum_grp.groupby('player_name').cumsum()

        df_cum_last = df_cum_sum.copy()
        for col in range(df_cum_sum.shape[1]):
            df_cum_last.iloc[:, col] = df_cum_sum.iloc[:, col] - df_cum_grp.iloc[:, col + 3]
        df_cum_last.columns = ['cum_each_game_score', 'cum_player_kills', 'cum_player_dmg', 'cum_player_assists', 
                               'dmg_kills', 'ride_walk', 'cum_num']

        df_cum = pd.concat([df_cum_grp, df_cum_last], axis=1)
        df_cum_m = df_cum[['date', 'match_id', 'player_name', 'cum_each_game_score', 'cum_player_kills', 
                           'cum_player_dmg', 'cum_player_assists', 'cum_num']]

        df = pd.merge(df, df_cum_m, how='left', on=['date', 'match_id', 'player_name'])

        return df

    def match_level(self, df):
        match_player_level = df.groupby(['date', 'match_id', 'player_name'])['cum_each_game_score'].mean().reset_index()

        match_level = match_player_level.groupby(['date', 'match_id'])['cum_each_game_score'].mean().reset_index()
        match_level.columns = ['date', 'match_id', 'match_level']

        df = pd.merge(df, match_level, how='left', on=['date', 'match_id'])

        return df

    def team_level(self, df):
        team_player_level = df.groupby(['date', 'match_id', 'team_id', 'player_name'])['cum_each_game_score'].mean().reset_index()

        match_level = df.groupby(['date', 'match_id'])['cum_each_game_score'].sum().reset_index()
        match_level.columns = ['date', 'match_id', 'match_level_sum']

        team_level_sum = df.groupby(['date', 'match_id', 'team_id'])[['cum_each_game_score', 'cum_num']].sum().reset_index()
        team_level_sum.columns = ['date', 'match_id', 'team_id', 'team_level_sum', 'team_cum_num']

        team_level = pd.merge(team_level_sum, match_level, how='left', on=['date', 'match_id'])
        team_level['team_level'] = team_level['team_level_sum'] / team_level['match_level_sum']

        df = pd.merge(df, team_level, how='left', on=['date', 'match_id', 'team_id'])

        return df

    def first_kill_time(self, df):
        first_kill_time = df.groupby(['date', 'match_id', 'team_id'])[['time']].min().reset_index()
        first_kill_time.columns = ['date', 'match_id', 'team_id', 'first_kill_time']
        df = pd.merge(df, first_kill_time, how='left', on=['date', 'match_id', 'team_id'])


        def get_team_tendency(first_kill_time):
            if first_kill_time <= 125:
                return 6
            elif first_kill_time <= 250:
                return 5
            elif first_kill_time <= 375:
                return 4
            elif first_kill_time <= 500:
                return 3
            elif first_kill_time <= 625:
                return 2
            elif first_kill_time <= 750:
                return 1
            else:
                return 0

        df['tendency'] = df['first_kill_time'].apply(lambda x: get_team_tendency(x))

        return df

    def kill_position(self, df):

        # 강북
        df.loc[((df['killer_position_x'] > 120000) & (df['killer_position_x'] < 240000)) 
           & ((df['killer_position_y'] > 550000) & (df['killer_position_y'] < 600000)), '강북'] = 1
        df['강북'] = df['강북'].fillna(0)

        # 강남
        df.loc[(df['killer_position_x'] > 100000) & (df['killer_position_x'] < 220000)
               & (df['killer_position_y'] > 480000) & (df['killer_position_y'] < 540000), '강남'] = 1
        df['강남'] = df['강남'].fillna(0)

        # 포친키
        df.loc[((df['killer_position_x'] > 330000) & (df['killer_position_x'] < 380000))
               & ((df['killer_position_y'] > 380000) & (df['killer_position_y'] < 420000)), '포친키'] = 1
        df['포친키'] = df['포친키'].fillna(0)

        # 야스나야
        df.loc[((df['killer_position_x'] > 500000) & (df['killer_position_x'] < 570000))
               & ((df['killer_position_y'] > 540000) & (df['killer_position_y'] < 600000)), '야스나야'] = 1
        df['야스나야'] = df['야스나야'].fillna(0)

        # 프리모스크
        df.loc[((df['killer_position_x'] > 140000) & (df['killer_position_x'] < 180000))
               & ((df['killer_position_y'] > 170000) & (df['killer_position_y'] < 220000)), '프리모스크'] = 1
        df['프리모스크'] = df['프리모스크'].fillna(0)

        # 밀리터리베이스
        df.loc[((df['killer_position_x'] > 520000) & (df['killer_position_x'] < 580000))
               &((df['killer_position_y'] > 160000) & (df['killer_position_y'] < 195000)), '밀리터리베이스'] = 1
        df['밀리터리베이스'] = df['밀리터리베이스'].fillna(0)

        # 밀타
        df.loc[((df['killer_position_x'] > 560000) & (df['killer_position_x'] < 610000))
               & ((df['killer_position_y'] > 310000) & (df['killer_position_y'] < 350000)), '밀타'] = 1
        df['밀타'] = df['밀타'].fillna(0)

        # 밀타파워
        df.loc[((df['killer_position_x'] > 700000) & (df['killer_position_x'] < 730000))
               & ((df['killer_position_y'] > 350000) & (df['killer_position_y'] < 400000)), '밀타파워'] = 1
        df['밀타파워'] = df['밀타파워'].fillna(0)

        # 노보(무역항)
        df.loc[((df['killer_position_x'] > 580000) & (df['killer_position_x'] < 630000))
               & ((df['killer_position_y'] > 180000) & (df['killer_position_y'] < 230000)), '노보'] = 1
        df['노보'] = df['노보'].fillna(0)

        # 리포브카
        df.loc[((df['killer_position_x'] > 680000) & (df['killer_position_x'] < 710000))
               & ((df['killer_position_y'] > 450000) & (df['killer_position_y'] < 490000)), '리포브카'] = 1
        df['리포브카'] = df['리포브카'].fillna(0)

        # 로족
        df.loc[((df['killer_position_x'] > 380000) & (df['killer_position_x'] < 420000))
               & ((df['killer_position_y'] > 500000) & (df['killer_position_y'] < 530000)), '로족'] = 1
        df['로족'] = df['로족'].fillna(0)

        # 자키
        df.loc[((df['killer_position_x'] > 100000) & (df['killer_position_x'] < 140000))
               & ((df['killer_position_y'] > 650000) & (df['killer_position_y'] < 690000)), '자키'] = 1
        df['자키'] = df['자키'].fillna(0)

        # 서버니
        df.loc[((df['killer_position_x'] > 360000) & (df['killer_position_x'] < 390000))
               & ((df['killer_position_y'] > 660000) & (df['killer_position_y'] < 690000)), '서버니'] = 1
        df['서버니'] = df['서버니'].fillna(0)

        # 스타버
        df.loc[((df['killer_position_x'] > 550000) & (df['killer_position_x'] < 580000))
               & ((df['killer_position_y'] > 660000) & (df['killer_position_y'] < 680000)), '스타버'] = 1
        df['스타버'] = df['스타버'].fillna(0)

        # 스쿨
        df.loc[((df['killer_position_x'] > 410000) & (df['killer_position_x'] < 440000))
               & ((df['killer_position_y'] > 460000) & (df['killer_position_y'] < 490000)), '스쿨'] = 1
        df['스쿨'] = df['스쿨'].fillna(0)

        # 통곡의다리1
        df.loc[((df['killer_position_x'] > 320000) & (df['killer_position_x'] < 370000))
               & ((df['killer_position_y'] > 210000) & (df['killer_position_y'] < 290000)), '통곡의다리1'] = 1
        df['통곡의다리1'] = df['통곡의다리1'].fillna(0)

        # 통곡의다리2
        df.loc[((df['killer_position_x'] > 530000) & (df['killer_position_x'] < 560000))
               & ((df['killer_position_y'] > 200000) & (df['killer_position_y'] < 310000)), '통곡의다리2'] = 1
        df['통곡의다리2'] = df['통곡의다리2'].fillna(0)

        return df

## 파생변수2 생성

In [15]:
import datetime

In [22]:
df = df.sort_values(['date', 'match_id'])

In [23]:
var = DerivingVariables2()

df = var.date_vars(df)
df = var.weapon_category(df)
df = var.dmg_per_kills(df)
df = var.ride_ratio(df)
df = var.cumulative_sum(df)
df = var.match_level(df)
df = var.team_level(df)
df = var.first_kill_time(df)
df = var.kill_position(df)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36231117 entries, 0 to 36231116
Data columns (total 68 columns):
 #   Column               Dtype              
---  ------               -----              
 0   date                 datetime64[ns, UTC]
 1   game_size            int64              
 2   match_id             object             
 3   party_size           int64              
 4   player_assists       int64              
 5   player_dbno          int64              
 6   player_dist_ride     float64            
 7   player_dist_walk     float64            
 8   player_dmg           int64              
 9   player_kills         int64              
 10  player_name          object             
 11  player_survive_time  float64            
 12  team_id              int64              
 13  team_placement       int64              
 14  killed_by            object             
 15  killer_placement     float64            
 16  killer_position_x    float64            
 17  killer

In [25]:
df.head()

,date,game_size,match_id,party_size,player_assists,player_dbno,player_dist_ride,player_dist_walk,player_dmg,player_kills,...,밀타파워,노보,리포브카,로족,자키,서버니,스타버,스쿨,통곡의다리1,통곡의다리2
0,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,4,0,0,0.000,213.061,0,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,4,1,0,3255.289,2690.863,75,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,4,0,2,5162.305,1682.161,236,3,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,4,0,2,5162.305,1682.161,236,3,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,4,0,2,5162.305,1682.161,236,3,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [26]:
pd.set_option('display.max_columns', None)

In [27]:
df.head()

,date,game_size,match_id,party_size,player_assists,player_dbno,player_dist_ride,player_dist_walk,player_dmg,player_kills,player_name,player_survive_time,team_id,team_placement,killed_by,killer_placement,killer_position_x,killer_position_y,map,time,victim_name,victim_placement,victim_position_x,victim_position_y,outlier,kill_dist,max_placement,each_game_score,total_score,kda,num_of_match,day_of_week,weekend,date_time,time_zone,weapon,dmg_per_kills,ride_ratio,kor_date,cum_each_game_score,cum_player_kills,cum_player_dmg,cum_player_assists,cum_num,match_level,team_level_sum,team_cum_num,match_level_sum,team_level,first_kill_time,tendency,강북,강남,포친키,야스나야,프리모스크,밀리터리베이스,밀타,밀타파워,노보,리포브카,로족,자키,서버니,스타버,스쿨,통곡의다리1,통곡의다리2
0,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,4,0,0,0.000,213.061,0,0,ZzyuanH,246.816,3,12,NaN,NaN,NaN,NaN,ERANGEL,NaN,NaN,NaN,NaN,NaN,0.000,0.000,24,1.200,6.524,0.000,2,4,0,17,evening,NaN,NaN,0.000,2017-10-20 17:01:07+00:00,0.000,0,0,0,0,0.000,0.000,0,0.000,NaN,584.000,2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,4,1,0,3255.289,2690.863,75,0,UnuSual13,1282.921,3,12,NaN,NaN,NaN,NaN,ERANGEL,NaN,NaN,NaN,NaN,NaN,0.000,0.000,24,5.531,5.531,1.000,1,4,0,17,evening,NaN,0.001,1.210,2017-10-20 17:01:07+00:00,0.000,0,0,0,0,0.000,0.000,0,0.000,NaN,584.000,2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,4,0,2,5162.305,1682.161,236,3,tsrvenom,1255.673,3,12,AKM,12.000,245371.600,468524.600,ERANGEL,1093.000,TxxCxx,14.000,245214.200,467492.000,0.000,1044.527,24,6.468,292.700,0.671,78,4,0,17,evening,AKM,78.667,3.069,2017-10-20 17:01:07+00:00,0.000,0,0,0,0,0.000,0.000,0,0.000,NaN,584.000,2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,4,0,2,5162.305,1682.161,236,3,tsrvenom,1255.673,3,12,AKM,12.000,246427.200,466539.400,ERANGEL,1046.000,dkswjs333,14.000,246326.700,465711.800,0.000,833.680,24,6.468,292.700,0.671,78,4,0,17,evening,AKM,78.667,3.069,2017-10-20 17:01:07+00:00,0.000,0,0,0,0,0.000,0.000,0,0.000,NaN,584.000,2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,4,0,2,5162.305,1682.161,236,3,tsrvenom,1255.673,3,12,S686,12.000,549529.400,595979.300,ERANGEL,584.000,CALLTOBO-MATING,19.000,549063.600,595732.600,0.000,527.096,24,6.468,292.700,0.671,78,4,0,17,evening,S686,78.667,3.069,2017-10-20 17:01:07+00:00,0.000,0,0,0,0,0.000,0.000,0,0.000,NaN,584.000,2,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [28]:
df_var2 = df.copy()

# Column 정리

In [31]:
df.columns

Index(['date', 'game_size', 'match_id', 'party_size', 'player_assists',
       'player_dbno', 'player_dist_ride', 'player_dist_walk', 'player_dmg',
       'player_kills', 'player_name', 'player_survive_time', 'team_id',
       'team_placement', 'killed_by', 'killer_placement', 'killer_position_x',
       'killer_position_y', 'map', 'time', 'victim_name', 'victim_placement',
       'victim_position_x', 'victim_position_y', 'outlier', 'kill_dist',
       'max_placement', 'each_game_score', 'total_score', 'kda',
       'num_of_match', 'day_of_week', 'weekend', 'date_time', 'time_zone',
       'weapon', 'dmg_per_kills', 'ride_ratio', 'kor_date',
       'cum_each_game_score', 'cum_player_kills', 'cum_player_dmg',
       'cum_player_assists', 'cum_num', 'match_level', 'team_level_sum',
       'team_cum_num', 'match_level_sum', 'team_level', 'first_kill_time',
       'tendency', '강북', '강남', '포친키', '야스나야', '프리모스크', '밀리터리베이스', '밀타', '밀타파워',
       '노보', '리포브카', '로족', '자키', '서버니', '스타버', '스쿨', '

In [38]:
del_cols = ['party_size', 'player_name', 'killed_by', 'killer_position_x', 'killer_position_y', 'map', 'time', 'victim_name', 'victim_placement', 'victim_position_x', 'victim_position_y']

In [39]:
for col in del_cols:
    del df[col]

In [40]:
df.head()

,date,game_size,match_id,player_assists,player_dbno,player_dist_ride,player_dist_walk,player_dmg,player_kills,player_survive_time,team_id,team_placement,killer_placement,outlier,kill_dist,max_placement,each_game_score,total_score,kda,num_of_match,day_of_week,weekend,date_time,time_zone,weapon,dmg_per_kills,ride_ratio,kor_date,cum_each_game_score,cum_player_kills,cum_player_dmg,cum_player_assists,cum_num,match_level,team_level_sum,team_cum_num,match_level_sum,team_level,first_kill_time,tendency,강북,강남,포친키,야스나야,프리모스크,밀리터리베이스,밀타,밀타파워,노보,리포브카,로족,자키,서버니,스타버,스쿨,통곡의다리1,통곡의다리2
0,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,0,0,0.000,213.061,0,0,246.816,3,12,NaN,0.000,0.000,24,1.200,6.524,0.000,2,4,0,17,evening,NaN,NaN,0.000,2017-10-20 17:01:07+00:00,0.000,0,0,0,0,0.000,0.000,0,0.000,NaN,584.000,2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,1,0,3255.289,2690.863,75,0,1282.921,3,12,NaN,0.000,0.000,24,5.531,5.531,1.000,1,4,0,17,evening,NaN,0.001,1.210,2017-10-20 17:01:07+00:00,0.000,0,0,0,0,0.000,0.000,0,0.000,NaN,584.000,2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,0,2,5162.305,1682.161,236,3,1255.673,3,12,12.000,0.000,1044.527,24,6.468,292.700,0.671,78,4,0,17,evening,AKM,78.667,3.069,2017-10-20 17:01:07+00:00,0.000,0,0,0,0,0.000,0.000,0,0.000,NaN,584.000,2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,0,2,5162.305,1682.161,236,3,1255.673,3,12,12.000,0.000,833.680,24,6.468,292.700,0.671,78,4,0,17,evening,AKM,78.667,3.069,2017-10-20 17:01:07+00:00,0.000,0,0,0,0,0.000,0.000,0,0.000,NaN,584.000,2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,2017-10-20 08:01:07+00:00,24,2U4GBNA0Yml6KV2X5zKbFWJzGxFNOvOB8zq4gwaVkAyf_p...,0,2,5162.305,1682.161,236,3,1255.673,3,12,12.000,0.000,527.096,24,6.468,292.700,0.671,78,4,0,17,evening,S686,78.667,3.069,2017-10-20 17:01:07+00:00,0.000,0,0,0,0,0.000,0.000,0,0.000,NaN,584.000,2,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


## csv로 내보내기

In [41]:
df.to_csv('top_10.csv', index=False)

In [ ]:
df.info()